In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Graph Convolutional Networks (GCN)

Baseado em: https://antonsruberts.github.io/graph/gcn/
        
No geral, metodos para extracao de caracteristicas nao capturam informacoes sobre a vizinhanca de um no em um grafo, as quais podem ser essenciais para alguns tipos de tarefas. Redes neurais em grafos resolvem esse problema ao considerar tanto a informacao de cada amostra como das suas relacoes com o restante do grafo para extrair essas caracteristicas. Como o nome sugere, GCN e uma rede neural que trabalha com dados em grafos, e o seu principal objetivo e refinar as informacoes presentes tanto no grafo em si como nos atributos de cada no em um vetor de caracteristicas. A figura abaixo apresenta uma ideia intuitiva da GCN de [Kipf and Welling (2016)](https://arxiv.org/abs/1609.02907). 
![](https://drive.google.com/uc?export=view&id=1liHJYh4Dte88zRPzjkWuk74UNrov5xPy)

Nessa aula, vamos usar o pacote `stellargraph` [(docs)](https://stellargraph.readthedocs.io/en/stable/) e sua implementacao de GCN. Os autores fornecem um bom exemplo de aplicacao usando jupyter [aqui](https://stellargraph.readthedocs.io/en/stable/demos/node-classification/gcn-node-classification.html).

O objetivo da aula e proporcionar uma boa ideia do que acontece em cada camada da rede e como aplicar esse modelo para datasets reais.

## Importando os pacotes

In [2]:
!pip install stellargraph
!pip uninstall umap -y
!pip install umap-learn

In [ ]:
import pandas as pd
from tqdm import tqdm
import json
import os
import umap.umap_ as umap
import numpy as np
import scipy.sparse as sp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score, confusion_matrix


import stellargraph as sg
from stellargraph.mapper import FullBatchNodeGenerator
from stellargraph.layer import GCN

import warnings
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import activations, initializers, constraints, regularizers
from tensorflow.keras.layers import Input, Layer, Lambda, Dropout, Reshape, Dense
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras import layers, optimizers, losses, metrics, Model
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Dados
---

Vamos comecar baixando o dataset de usuarios do github, disponivel [aqui](https://snap.stanford.edu/data/github-social.html). Usaremos o dataset para classificar se o usuario de desenvolvedor web ou de ML. Nesse dataset, os nos representam usuarios do github que deram estrela para mais de 10 repositorios, as arestas representam seguidores em comum, e as caracteristicas representam a localizacao, repositorios com estrelas, empregador e email.

### Baixando e extraindo o dataset

In [ ]:
!wget "https://snap.stanford.edu/data/git_web_ml.zip"
!unzip git_web_ml.zip

### Lendo as arestas, caracteristicas e rotulos

In [ ]:
edges_path = '/content/drive/MyDrive/Colab Notebooks/Aula GCN/git_web_ml/musae_git_edges.csv'
targets_path = '/content/drive/MyDrive/Colab Notebooks/Aula GCN/git_web_ml/musae_git_target.csv'
features_path = '/content/drive/MyDrive/Colab Notebooks/Aula GCN/git_web_ml/musae_git_features.json'

### definindo um numero menor de amostras

Farei isso para treinar mais rapidamente e evitar erro de crash de memoria 

In [ ]:
num_samples = 10000

In [3]:
# lendo as arestas
edges = pd.read_csv(edges_path)
edges.columns = ['source', 'target'] # renaming for StellarGraph compatibility
display(edges.shape, edges.head())

In [4]:
edges = edges[edges['target'] <num_samples]
edges = edges[edges['source'] <num_samples]
display(edges.shape, edges.head())

In [5]:
# lendo as caracteristicas
with open(features_path) as json_data:
    features = json.load(json_data)

max_feature = np.max([v for v_list in features.values() for v in v_list])
# features_matrix = np.zeros(shape = (len(list(features.keys())), max_feature+1))
features_matrix = np.zeros(shape = (num_samples, max_feature+1))


i = 0
for k, vs in tqdm(features.items()):
    if i<num_samples:
        for v in vs:
            features_matrix[i, v] = 1
        i+=1

In [6]:
# node_features = pd.DataFrame(features_matrix, index = features.keys())
node_features = pd.DataFrame(features_matrix, index = np.arange(num_samples).astype(str))
display(node_features.shape, node_features.head())

In [7]:
# lendo os rotulos
targets = pd.read_csv(targets_path)
targets.index = targets.id.astype(str)
targets = targets.loc[np.arange(num_samples).astype(str), :]
display(targets.shape, targets.head(), targets.ml_target.value_counts(normalize=True))

Entao temos 37700 desenvolvedores, 289003 seguidores em comum e 4005 caracteristicas. Aproximadamente 75% dos usuarios sao desenvolvedores web e 25% sao desenvolvedores de ML.

### Dados no StellarGraph

`stellargraph` tem sua propria estrutura de dados com diversas funcionalidades. Para transformar nossos dados para o formato do StellarGraph e muito simples, precisando apenas passar os dataframes com as caracteristicas dos nos e as arestas para a funcao `StellarGraph`. A API tambem aceita arestas ponderadas, nos heterogeneos, tipos de arestas e nos direcionados.

In [8]:
G = sg.StellarGraph(node_features, edges.astype(str))
print(G.info())

### Conjuntos de Treino, Teste e validacao

Como de costume, precisamos separar os dados em treino, validacao e teste. GCN e um modelo semi-supervisionado, o que significa que precisa de um numero reduzido de rotulos do que tecnicas supervisionadas para o aprendizado. Suponha que tenhamos apenas 1% dos dados rotulados, i.e., 400 desenvolvedores. Nesse caso, usaremmos 200 deles para treinamento e os outros 200 para validacao. O restante sera usado para teste.

In [ ]:
train_pages, test_pages = train_test_split(targets, train_size=200)
val_pages, test_pages = train_test_split(test_pages, train_size=200)

In [9]:
train_pages.shape, val_pages.shape, test_pages.shape

## Pre-processamento

### Pre-processamento dos rotulos

In [ ]:
target_encoding = LabelBinarizer()

train_targets = target_encoding.fit_transform(train_pages['ml_target'])
val_targets = target_encoding.transform(val_pages['ml_target'])
test_targets = target_encoding.transform(test_pages['ml_target'])

# print(train_pages['ml_target'])
# print(train_targets)

### Pre-processamento do grafo

Essa e a principal parte para o funcionamento de GCN. Para entender o tipo de processamento que faremos, vamos das uma olhada no que as camadas convolucionais em grafo fazem. O que queremos e, de alguma forma, agregar a informacao das caracteristicas dos nos vizinhos porque queremos aprender as representacoes que reflitam a vizinhanca do grafo. Em CNNs, que sao normalmente usadas para imagens, esse objetivo e alcancado usando operacoes convolucionais com pixels e kernels. A intensidade do no (pixel) vizinho (ex: 3x3) e passada pelo kernel que tira a media dos pixels e computa um unico valor. Isso funciona porque em imagens os vizinhos sao ordenados e tem tamanho fixo. Nos nao temos essas qualidades em grafos, e precisamos de uma alternativa diferente.



![](https://drive.google.com/uc?export=view&id=1LWb4Lc4Nhqzm0GteBM1jZ-0ib4fjlRBQ)

A alternativa e usar a ideia de passar a infomacao multiplicando os estados ocultos pela matriz de adjacencia. A matriz de adjacencia representa a conexao entre os nos. Sendo assim, multiplicando os estados ocultos (ou os nos com as caracteristicas, para a primeira camada) por ela, estamos de certa forma aplicando uma mascara e agregando apenas a informacao dos nos vizinhos. Os conceitos podem ser vistos em detalhe nesse [video](https://www.youtube.com/watch?v=ijmxpItkRjc&t=524s).

Nossa tarefa agora e pre-computar essa parte nao treinavel do algoritmo. Usaremos a implementacao do stellargraph, que executa esses calculos em um formato esparco para melhorar a velocidade.

In [10]:
# Pega a matriz de adjacencias do grafo
A = G.to_adjacency_matrix(weighted=False)

# Adiciona conexoes dos nos com eles mesmos (se a representacao seria baseada apenas nos vizinhos, sem considerar as caracteristicas do proprio no)
A_t = A + sp.diags(np.ones(A.shape[0]) - A.diagonal())

# Computa o grau da matriz e ele a -1/2 (grau e o numero de vizinhos, e eleva a -1/2 para ajudar na normalizacao)
D_t = sp.diags(np.power(np.array(A.sum(1)), -0.5).flatten(), 0)

# Normaliza a matriz de adjacencias
A_norm = A.dot(D_t).transpose().dot(D_t).todense()

Agora que pre-processamos nossos dados para o GCN, devemos seguir mais algumas formalidades antes de treinar o modelo:
1. Pegar os novos indices dos conjuntos de treinamento, validacao e teste - necessarios para computar o custo do modelo.
2. Adicionar uma dimensao extra para os nossos dados (necessario pela implementacao das redes - pense como se fosse um minibatch com uma unica amostra)

In [ ]:
# Definindo uma funcao para pegar esses indices
def get_node_indices(G, ids):
    # encontra os indices dos nos
    node_ids = np.asarray(ids)
    flat_node_ids = node_ids.reshape(-1)

    flat_node_indices = G.node_ids_to_ilocs(flat_node_ids) # usa funcao in-built 
    # volta para o formato original
    node_indices = flat_node_indices.reshape(1, len(node_ids)) # adiciona uma dimensao extra
    
    return node_indices

train_indices = get_node_indices(G, train_pages.index)
val_indices = get_node_indices(G, val_pages.index)
test_indices = get_node_indices(G, test_pages.index)

In [ ]:
# Expandindo para a  dimensao extra
features_input = np.expand_dims(features_matrix, 0)
A_input = np.expand_dims(A_norm, 0)

y_train = np.expand_dims(train_targets, 0)
y_val = np.expand_dims(val_targets, 0)
y_test = np.expand_dims(test_targets, 0)

Agora que os dados estao normalizados e no formato correto, podemos comecar a modelagem da rede.

## Modelo GCN

Analisando friamente, as camadas GCN nao passam de multiplicacoes de valores de entrada, pesos, e a matriz de adjacencias normalizada. Quem tiver interesse, pode ver como e feita implementacao em `stellargraph`'s `GraphConvolution` [aqui](https://github.com/stellargraph/stellargraph/blob/develop/stellargraph/layer/gcn.py), nas linhas 208 e 209.

In [ ]:
from stellargraph.layer.gcn import GraphConvolution, GatherIndices

# Inicializando os parametros da GCN
kernel_initializer="glorot_uniform"
bias = True
bias_initializer="zeros"
n_layers = 2
layer_sizes = [32, 32]
dropout = 0.5
n_features = features_input.shape[2]
n_nodes = features_input.shape[1]

Primeiramente, vamos inicializar a camada de entrada com os formatos corretos para receber 3 entradas:
1. Matriz de caracteristicas
2. Indices de treinamento/teste/validacao
3. Matriz de adjacencias normalizada

In [11]:
x_features = Input(batch_shape=(1, n_nodes, n_features))
x_indices = Input(batch_shape=(1, None), dtype="int32")
x_adjacency = Input(batch_shape=(1, n_nodes, n_nodes))
x_inp = [x_features, x_indices, x_adjacency]
x_inp

Agora, podemos construir um modelo GCN com 2 camadas e dropout. Cada camada vai ter 32 unidades, que devem ser suficientes para transformar os dados o obter boas representacoes.

In [12]:
x = Dropout(0.5)(x_features)

x = GraphConvolution(32, activation='relu', 
                     use_bias=True,
                     kernel_initializer=kernel_initializer,
                     bias_initializer=bias_initializer)([x, x_adjacency])
x = Dropout(0.5)(x)
x = GraphConvolution(32, activation='relu', 
                     use_bias=True,
                     kernel_initializer=kernel_initializer,
                     bias_initializer=bias_initializer)([x, x_adjacency])

x = GatherIndices(batch_dims=1)([x, x_indices])
output = Dense(1, activation='sigmoid')(x)

In [13]:
model = Model(inputs=[x_features, x_indices, x_adjacency], outputs=output)
model.summary()

In [ ]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.01),
    loss=losses.binary_crossentropy,
    metrics=["acc"],
)

In [ ]:
es_callback = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

In [14]:
history = model.fit(
    x = [features_input, train_indices, A_input],
    y = y_train,
    batch_size = 32,
    epochs=200,
    validation_data=([features_input, val_indices, A_input], y_val),
    verbose=1,
    shuffle=False,
    callbacks=[es_callback],
)

## Avaliando o modelo

Agora que treinamos o modelo, vamos avaliar sua acuracia sobre o conjunto de teste.

In [ ]:
test_preds = model.predict([features_input, test_indices, A_input])

In [ ]:
def evaluate_preds(true, pred):
    auc = roc_auc_score(true, pred)
    pr = average_precision_score(true, pred)
    bin_pred = [1 if p > 0.5 else 0 for p in pred]
    f_score = f1_score(true, bin_pred)
    print('ROC AUC:', auc)
    print('PR AUC:', pr)
    print('F1 score:', f_score)
    print(confusion_matrix(true, bin_pred, normalize='true'))
    
    return auc, pr, f_score

In [15]:
auc, pr, f_score = evaluate_preds(test_targets.ravel(),test_preds[0].ravel())

Temos uma pontuacao ROC AUC com 0.80, mas que poderia ser melhorada para aproximadamente 0.89 se nao fosse o corte brusco no dataset, assim como as demais metricas. Vamos visualizar como ficaram as representacoes.

In [16]:
embedding_model = Model(inputs=x_inp, outputs=model.layers[-2].output)
all_indices = get_node_indices(G, targets.index)
emb = embedding_model.predict([features_input, all_indices, A_input])
emb.shape

In [17]:
u = umap.UMAP(random_state=42)
umap_embs = u.fit_transform(emb[0])

In [18]:
plt.figure(figsize=(20,10))
ax = sns.scatterplot(x = umap_embs[:, 0], y = umap_embs[:, 1], hue = targets['ml_target'])

No plot conseguimos identificar que o algoritmo conseguiu separar razoavelmente bem 2 classes, mas deixou um cluster de fora. Provavelmente, esse cluster extra foi criado devido as amostras e arestas que cortamos para economizar memoria.

## Conclusoes
---

GCNs sao redes neurais poderosas que permitem combinar informacoes obtidas a partir das caracteristicas dos nos e da vizinhanca. Tal propriedade e alcancada multiplicando as saidas das camadas anteriores pela matriz de adjacencia normalizada, o que performa uma operacao similar a um filtro de convolucao. As caracteristicas dos filtros vizinhos sao agregadas e se tornam representacoes muito uteis a serem aprendidas e usadas para back-propagation, como de costume.

## Exercicios

1. Arranjar o dataset de modo que exclua todas as amostras com ID maior que 10000, e que nao fique nenhum no que nao tenha nenhum vizinho (todos os vizinhos foram excluidos). Testar se melhora os resultados do modelo.